**Общая информация**

**Срок сдачи:** 21 октября 2019, 08:30 

**Штраф за опоздание:** по 0.5 балла за 24 часа задержки. Через 10 дней домашнее задание сгорает.

При отправлении ДЗ указывайте фамилию в названии файла
Присылать ДЗ необходимо в виде ссылки на свой github репозиторий на почту ml1.sphere@mail.ru с указанием темы в следующем формате:

[ML0919, Задание 1] Фамилия Имя.

Используйте данный Ipython Notebook при оформлении домашнего задания.


**Штрафные баллы:**

1. Невыполнение PEP8 -0.5 баллов
2. Отсутствие фамилии в имени скрипта (скрипт должен называться по аналогии со stroykova_hw1.ipynb) -0.5 баллов
3. Все строчки должны быть выполнены. Нужно, чтобы output команды можно было увидеть уже в git'е. В противном случае -0.5 баллов


In [1]:
import numpy as np
import matplotlib.pyplot as plt

import seaborn as sns
from sklearn import datasets
from sklearn.base import ClassifierMixin
from sklearn.datasets import fetch_mldata, fetch_20newsgroups

from sklearn.neighbors.base import NeighborsBase, KNeighborsMixin, SupervisedIntegerMixin 
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
%load_ext pycodestyle_magic

##### Задание 1 (1 балл)
Реализовать KNN в классе MyKNeighborsClassifier (обязательное условие: точность не ниже sklearn реализации)
Разберитесь самостоятельно, какая мера расстояния используется в KNeighborsClassifier дефолтно и реализуйте свой алгоритм именно с этой мерой. Самостоятельно разберитесь, как считается score из KNeighborsClassifier и реализуйте аналог в своём классе. Score не должен уступать значению KNN из sklearn

In [2]:
#%%pycodestyle
from collections import Counter
from sklearn.neighbors import KDTree
from functools import reduce
from sklearn.metrics.pairwise import cosine_similarity


class MyKNeighborsClassifier(NeighborsBase, KNeighborsMixin,
                             SupervisedIntegerMixin, ClassifierMixin):

    def get_response(self, X):  # получает предсказание для точки
        if self.algorithm == 'kd_tree':
            _, idx = self.tree.query(X[np.newaxis, :], self.n_neighbors)
            neighb_idx = idx[0]
        else:
            X1 = np.array(X)[np.newaxis, :]
            X1 = np.repeat(X1, self.X.shape[0], axis=0)
            dist = np.linalg.norm(X1 - self.X[:, :-1], axis=1)
            neighb_idx = np.argsort(dist)[:self.n_neighbors]
        pred = np.array([self.X[j][-1] for j in neighb_idx])
        return Counter(pred).most_common()

    def __init__(self, n_neighbors, algorithm='brute'):
        self.n_neighbors = n_neighbors
        self.algorithm = algorithm

    def fit(self, X, y):
        self.X = np.concatenate((np.array(X),
                                 np.array(y)[np.newaxis,:].reshape(-1, 1)), axis=1)
        if self.algorithm == 'kd_tree':
            self.tree = KDTree(self.X[:, :-1], leaf_size=2)

    def predict(self, X):
        return np.array([self.get_response(X[i])[0][0]
                         for i in range(X.shape[0])], dtype='int')

    def predict_proba(self, X):
        classes = np.unique(self.X[:, -1])
        for i in range(X.shape[0]):
            cnt = self.get_response(X[i])
            num = list(filter(lambda x: x[1] == cnt[0][1], cnt))
            if i == 0:
                res = np.array(reduce(lambda c,
                                      x: np.logical_or(c, classes == x[0]),
                                      num, np.zeros_like(classes)),
                               dtype='float') / len(num)
                res = res[np.newaxis, :]
            else:
                res = np.concatenate((res, np.array(
                    reduce(lambda c,
                           x: np.logical_or(c, classes == x[0]), num,
                           np.zeros_like(classes)), dtype='float')
                                      [np.newaxis, :] / len(num)), axis=0)
        return res

    def score(self, X, y):
        # Returns the mean accuracy on the given test data and labels.
        y_test = self.predict(X)
        return (y_test == y).sum() / float(y_test.shape[0])

**IRIS**

В библиотеке scikit-learn есть несколько датасетов из коробки. Один из них [Ирисы Фишера](https://ru.wikipedia.org/wiki/%D0%98%D1%80%D0%B8%D1%81%D1%8B_%D0%A4%D0%B8%D1%88%D0%B5%D1%80%D0%B0)

In [3]:
iris = datasets.load_iris()

In [4]:
X_train, X_test, y_train, y_test = train_test_split(iris.data, iris.target, test_size=0.1, stratify=iris.target)

In [5]:
clf = KNeighborsClassifier(n_neighbors=2, algorithm='brute')
my_clf = MyKNeighborsClassifier(n_neighbors=2, algorithm='brute')

In [6]:
clf.fit(X_train, y_train)
my_clf.fit(X_train, y_train)

In [7]:
assert abs(my_clf.score(X_test, y_test) - clf.score(X_test,y_test))<0.005, "Score must be simillar"

**Задание 2 (0.5 балла)**

Давайте попробуем добиться скорости работы на fit, predict и predict_proba сравнимой со sklearn для iris.
Для этого используем numpy

In [8]:
%time clf.fit(X_train, y_train)

CPU times: user 1.37 ms, sys: 0 ns, total: 1.37 ms
Wall time: 1.4 ms


KNeighborsClassifier(algorithm='brute', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=2, p=2,
                     weights='uniform')

In [9]:
%time my_clf.fit(X_train, y_train)

CPU times: user 103 µs, sys: 12 µs, total: 115 µs
Wall time: 128 µs


In [10]:
%time clf.predict(X_test)

CPU times: user 6.88 ms, sys: 342 µs, total: 7.23 ms
Wall time: 6.41 ms


array([0, 1, 2, 1, 0, 0, 1, 2, 1, 2, 2, 0, 2, 0, 1])

In [11]:
%time my_clf.predict(X_test)

CPU times: user 5.38 ms, sys: 165 µs, total: 5.54 ms
Wall time: 5.47 ms


array([0, 1, 2, 1, 0, 0, 1, 2, 1, 2, 2, 0, 2, 0, 1])

In [12]:
%time clf.predict_proba(X_test)

CPU times: user 2.55 ms, sys: 3.83 ms, total: 6.38 ms
Wall time: 5.74 ms


array([[1., 0., 0.],
       [0., 1., 0.],
       [0., 0., 1.],
       [0., 1., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [0., 1., 0.],
       [0., 0., 1.],
       [0., 1., 0.],
       [0., 0., 1.],
       [0., 0., 1.],
       [1., 0., 0.],
       [0., 0., 1.],
       [1., 0., 0.],
       [0., 1., 0.]])

In [13]:
%time my_clf.predict_proba(X_test)

CPU times: user 11.1 ms, sys: 375 µs, total: 11.5 ms
Wall time: 10.3 ms


array([[1., 0., 0.],
       [0., 1., 0.],
       [0., 0., 1.],
       [0., 1., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [0., 1., 0.],
       [0., 0., 1.],
       [0., 1., 0.],
       [0., 0., 1.],
       [0., 0., 1.],
       [1., 0., 0.],
       [0., 0., 1.],
       [1., 0., 0.],
       [0., 1., 0.]])

###### Задание 3 (1 балл)
Добавьте algorithm='kd_tree' в реализацию KNN (использовать KDTree из sklearn.neighbors). Необходимо добиться скорости работы на fit,  predict и predict_proba сравнимой со sklearn для iris.
Для этого используем numpy. Score не должен уступать значению KNN из sklearn

In [14]:
clf = KNeighborsClassifier(n_neighbors=2, algorithm='kd_tree')
my_clf = MyKNeighborsClassifier(n_neighbors=2, algorithm='kd_tree')

In [15]:
X_train, X_test, y_train, y_test = train_test_split(iris.data, iris.target, test_size=0.1, stratify=iris.target)

In [16]:
%time clf.fit(X_train, y_train)

CPU times: user 2.64 ms, sys: 316 µs, total: 2.96 ms
Wall time: 2.35 ms


KNeighborsClassifier(algorithm='kd_tree', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=2, p=2,
                     weights='uniform')

In [17]:
%time my_clf.fit(X_train, y_train)

CPU times: user 0 ns, sys: 2.33 ms, total: 2.33 ms
Wall time: 1.85 ms


In [18]:
%time clf.predict(X_test)

CPU times: user 6.29 ms, sys: 3.8 ms, total: 10.1 ms
Wall time: 8.49 ms


array([2, 1, 2, 0, 2, 0, 0, 2, 1, 2, 2, 0, 1, 1, 0])

In [19]:
%time my_clf.predict(X_test)

CPU times: user 11.4 ms, sys: 0 ns, total: 11.4 ms
Wall time: 10 ms


array([2, 1, 2, 0, 2, 0, 0, 2, 1, 2, 2, 0, 1, 1, 0])

In [20]:
%time clf.predict_proba(X_test)

CPU times: user 8.21 ms, sys: 22 µs, total: 8.23 ms
Wall time: 6.04 ms


array([[0., 0., 1.],
       [0., 1., 0.],
       [0., 0., 1.],
       [1., 0., 0.],
       [0., 0., 1.],
       [1., 0., 0.],
       [1., 0., 0.],
       [0., 0., 1.],
       [0., 1., 0.],
       [0., 0., 1.],
       [0., 0., 1.],
       [1., 0., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       [1., 0., 0.]])

In [21]:
%time my_clf.predict_proba(X_test)

CPU times: user 11.3 ms, sys: 0 ns, total: 11.3 ms
Wall time: 11.2 ms


array([[0., 0., 1.],
       [0., 1., 0.],
       [0., 0., 1.],
       [1., 0., 0.],
       [0., 0., 1.],
       [1., 0., 0.],
       [1., 0., 0.],
       [0., 0., 1.],
       [0., 1., 0.],
       [0., 0., 1.],
       [0., 0., 1.],
       [1., 0., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       [1., 0., 0.]])

In [22]:
assert abs(my_clf.score(X_test, y_test) - clf.score(X_test,y_test))<0.005, "Score must be simillar"

**Задание 4 (2.5 балла)**

Рассмотрим новый датасет 20 newsgroups

In [23]:
newsgroups = fetch_20newsgroups(subset='train',remove=['headers','footers', 'quotes'])

In [24]:
data = newsgroups['data']
target = newsgroups['target']

Переведите во всех документах все буквы в нижний регистр и замените во всех документах символы, не
являющиеся буквами и цифрами, на пробелы. Далее разбейте текста по пробельным символам на токены(термы/слова). Удалите текста, содержащие только пробелы.

In [25]:
# data_tok should be a list of lists of tokens for each line in data.
import re
data_tok = list(map(lambda x: x.lower(), data))
data_tok = list(map(lambda x: re.compile(r"\W").sub(' ', x), data_tok))
data_tok = list(map(lambda x: x.split(), data_tok))
data_tok = list(filter(lambda x: x != [], data_tok))

In [26]:
assert all(isinstance(row, (list, tuple)) for row in data_tok), "please convert each line into a list of tokens (strings)"
assert all(all(isinstance(tok, str) for tok in row) for row in data_tok), "please convert each line into a list of tokens (strings)"
is_latin = lambda tok: all('a' <= x.lower() <= 'z' for x in tok)
assert all(map(lambda l: not is_latin(l) or l.islower() , map(' '.join, data_tok))), "please make sure that you lowercase the data and drop spaced texts"

Преобразуйте датасет в разреженную матрицу scipy.sparse.csr_matrix, где значение x в позиции (i, j)
означает, что в документе i слово j встретилось x раз

In [27]:
from scipy.sparse import csr_matrix
indptr = [0]
indices = []
data = []
vocabulary = {}
for d in data_tok:
    for term in d:
        index = vocabulary.setdefault(term, len(vocabulary))
        indices.append(index)
        data.append(1)
    indptr.append(len(indices))
csr_m = csr_matrix((data, indices, indptr), dtype=int).todense()

Остальное сделать не получилось, либо не хватает оперативной памяти, либо что-то не работает

*Так мы получили векторное представление наших текстов. Значит можно приступать к задаче обучения модели*

Реализуйте разбиение выборки для кросс-валидации на 3 фолдах. Разрешено использовать sklearn.cross_validation

In [28]:
from sklearn.model_selection import KFold
kf = KFold(n_splits=3)

Напишите метод, позволяющий найти оптимальное количество ближайших соседей(дающее максимальный score в среднем на валидации на 3 фолдах).
Постройте график зависимости среднего score от количества соседей. Можно рассмотреть число соседей от 1 до 10.

In [ ]:
clf = MyKNeighborsClassifier(n_neighbors=5, algorithm='kd_tree')
score = []
for train, test in kf.split(X):
    X_train, X_test = X[train], X[test]
    y_train, y_test = target[train], target[test]
    print(X_train.shape, y_train.shape)
    clf.fit(X_train, y_train)
    score.append(clf.score(X_test, y_test))
score_mean = sum(score) / len(score)
print(scores_mean)

Как изменится качество на валидации, если:

1. Используется косинусная метрика вместо евклидовой.
2. К текстам применяется TfIdf преобразование( sklearn.feature_extraction.text.TfidfTransformer)

Сравните модели, выберите лучшую.

Загрузим  теперь test  часть нашей выборки и преобразуем её аналогично с train частью. Не забудьте, что наборы слов в train и test части могут отличаться.

In [ ]:
newsgroups = fetch_20newsgroups(subset='test',remove=['headers','footers', 'quotes'])
test_data = newsgroups['data']
test_target = newsgroups['target']
test_tok = list(map(lambda x: x.lower(), test_data))
test_tok = list(map(lambda x: re.compile(r"\W").sub(' ', x), test_tok))
test_tok = list(map(lambda x: x.split(), test_tok))
test_tok = list(filter(lambda x: x != [], test_tok))

In [ ]:
indptr = [0]
indices = []
data = []
#old vocabulary
for d in test_tok:
    for term in d:
        if term in vocabulary.keys():
            index = vocabulary.setdefault(term, len(vocabulary))
            indices.append(index)
            data.append(1)
    indptr.append(len(indices))
test_csr = csr_matrix((data, indices, indptr), dtype=int, shape=(len(test_tok), 100)).todense()

Оценим качество(score) вашей лучшей модели на test части датасета. Отличается ли оно от кросс-валидации? Попробуйте сделать выводы, почему отличается качество.

In [ ]:
best_clf = MyKNeighborsClassifier(n_neighbors=5, algorithm='brute')
best_clf.fit(csr_m, target)
clf.score(test_csr, test_target)